In [1]:
!pip install replicate tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.1 MB/s eta 0:00:00


In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [10]:
!pip install -U langchain langchain-community langchain-experimental replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.4 MB/s eta 0:00:00


In [17]:
import replicate
from tqdm import tqdm
import os
import pandas as pd
import requests

In [5]:
from langchain_community.llms import Replicate
from google.colab import userdata
import os

# Ambil api token dari colab
api_token = userdata.get("api_token")

# Kita masukin token ke env var
os.environ["REPLICATE_API_TOKEN"] = api_token

In [7]:
from replicate.client import Client

replicate = Client(
  api_token=os.environ["REPLICATE_API_TOKEN"],
  headers={
    "User-Agent": "replicate-google-colab-example/1.0"
  }
)


In [19]:
df = pd.read_csv("/content/mtsamples.csv")
df.head(5)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [20]:
df = pd.read_csv("/content/mtsamples.csv")
df = df.iloc[:5].copy()


In [12]:

#  Bangun prompt untuk semua kolom
def build_prompt(row):
    return f"""
You are a medical data analyst. Based on the following structured medical information, provide:
1. Analytical Result
2. Insight & Findings
3. Recommendations

Data:
- Description: {row['description']}
- Medical Specialty: {row['medical_specialty']}
- Sample Name: {row['sample_name']}
- Transcription: {row['transcription']}
- Keywords: {row['keywords']}

Respond in structured format:
- Analytical Result:
- Insight & Findings:
- Recommendations:
"""

In [21]:
outputs = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    prompt = build_prompt(row)

    try:
        output = replicate.run(
            "ibm-granite/granite-3.3-8b-instruct",
            input={
                "prompt": prompt,
                "top_k": 5,
                "top_p": 1.0,
                "max_tokens": 4096,
                "temperature": 0.7,
                "repetition_penalty": 1.0
            }
        )
        response = "".join(output)
    except Exception as e:
        response = f"Error: {e}"

    outputs.append(response)

df["llm_output"] = outputs


100%|██████████| 5/5 [03:11<00:00, 38.34s/it]


In [22]:


df.to_csv("mtsamples_with_llm_output.csv", index=False)